In [1]:
from datetime import datetime
from IPython.display import display
# %reload_ext autotime

import pandas as pd
import numpy as np
import re
import joblib
import sys
#import pandas_profiling

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import mean_absolute_error
import os

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, mean_absolute_error, f1_score, log_loss

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

In [2]:
def basicDataExploration(df):
    print("General Stats::")
    print(df.info())

    print( 100*'-')


    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1] )

    print( 100*'-')

    print("Column Names:")
    print ( df.columns.values.tolist() )

    print( 100*'-')
    #Explore for missing values
    print("Columns with Missing Values:",df.columns[df.isnull().any()].tolist())
    
    #print("\nUnique Values Exploration: ")
    #features = df.columns.values
    #print (100*'-')
    #for feature in features:
        #unique_length =  len(df[str(feature)].unique() )
        #print(str(unique_length)+' Unique values for' + ' ' + f'"{feature}"' +': {}'.format(df[str(feature)].unique()))
        #print (100*'-')
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000,"display.max_columns", 1000): 
        display(df)
        
def null_summary( df ):
    # Number of null values in each feature
    null = df.isnull().sum()
    # Select those columns that contains null values
    null_counts = pd.DataFrame ( null.loc[ null> float(0) ] , columns =["Frequency"] )
    # Get the percentage of missing data for those features with missing values
    null_normalized = \
    null_counts.apply( lambda x: x/(df.shape[0] ) ).rename( columns={"Frequency":"Percentage"})
    # obtain a [Frequency, Percentage] dataframe to visualise the missing data
    null_df = pd.concat( [null_counts, null_normalized] , axis=1).sort_values(by="Percentage")
   
    null_df = null_df.style.background_gradient(cmap='Reds')

    return null_df

def my_value_counts( data, col):
    df= pd.concat( [pd.DataFrame(
        data[col].value_counts(dropna=False) ).rename(
    columns={col:"Frequency"}) 
                ,pd.DataFrame(
                data[col].value_counts(dropna=False, normalize = True)).rename(
                columns={col:"Proportion"})]
              ,axis=1 )
    
    df = df.append(
        pd.DataFrame(
        {"Frequency":[df["Frequency"].sum()],
         "Proportion":[df["Proportion"].sum()]},
        index=['Total'])
    )
    
    df = df.round(decimals=4)
    df = df.style.background_gradient(cmap='Blues')
    
    return df 

In [3]:
df_session_small = pd.read_csv('repeat_sessions.csv')
df_session_small

,full_visitor_id,client_id,session_id,quotation_id,visit_number,session_date,session_date_time,channelgrouping,source,medium,...,total_pa_pageviews,applicant_drop_out_flag,entrance_page_path,exit_page_path,entrance_page_title,exit_page_title,product_type,quote_issue_date,quote_saved_quote_indicator,quote_converted_indicator
0,4.736000e+18,GA1.3.1102685963.1629861488,4736000150473127536-1629861487,NaN,1,2021-08-25,2021-08-25 03:18:07,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/maid-insurance/,www.fwd.com.sg/maid-insurance/,Buy Maid Insurance Plan Online | FWD Singapore,Buy Maid Insurance Plan Online | FWD Singapore,NaN,NaN,NaN,NaN
1,2.904419e+18,GA1.3.676237732.1580681758,2904418944841894430-1616247384,ATYX4529860L6V1,2,2021-03-20,2021-03-20 13:36:24,Paid Search,google,cpc,...,0,0,www.fwd.com.sg/car-insurance/,www.fwd.com.sg/car-insurance/comprehensive-mot...,Buy Car Insurance Plan Online | FWD Singapore,Car Insurance Online Quote | FWD Singapore,Motor,2021-03-20 21:37:24.057,0.0,0.0
2,1.357610e+18,GA1.3.316093131.1628488004,1357609661763731780-1629983235,7HXO4880472DB74,2,2021-08-26,2021-08-26 13:07:15,(Other),CROSSSELL,EDM,...,0,0,www.fwd.com.sg/car-insurance/comprehensive-mot...,www.fwd.com.sg/car-insurance/comprehensive-mot...,Car Insurance Online Quote | FWD Singapore,Car Insurance Online Quote | FWD Singapore,Motor,2021-08-09 13:49:08.847,0.0,0.0
3,4.651399e+18,GA1.3.1082988166.1615619134,4651398756540638270-1615860154,NaN,3,2021-03-16,2021-03-16 02:02:34,(Other),Renewal,SMS,...,0,0,www.fwd.com.sg/maid-insurance/online-quote/ren...,www.fwd.com.sg/maid-insurance/online-quote/ren...,Maid Insurance Online Quote | FWD Singapore,Maid Insurance Online Quote | FWD Singapore,NaN,NaN,NaN,NaN
4,1.919897e+18,GA1.3.447010817.1624464357,1919896841597705189-1624464357,KZPU4748991SBMU,1,2021-06-24,2021-06-23 16:05:57,Social,m.facebook.com,referral,...,0,0,www.fwd.com.sg/commercial-car-insurance/,www.fwd.com.sg/commercial-car-insurance/,Buy Commercial Car Insurance Online | FWD Sing...,Buy Commercial Car Insurance Online | FWD Sing...,Motor,2021-06-24 00:09:37.947,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215042,4.605354e+18,GA1.3.1072267517.1614230247,4605353919692354279-1614566027,OXLU4482229NYTY,2,2021-03-01,2021-03-01 02:33:47,Organic Search,google,organic,...,0,0,www.fwd.com.sg/home-insurance/,www.fwd.com.sg/home-insurance/online-quote/dem...,Buy Home Insurance Online | FWD Singapore,Home Insurance Online Quote | FWD Singapore,Home,2021-03-01 10:45:00.91,0.0,1.0
1215043,3.555738e+18,GA1.3.827884798.1613353002,3555738133878919210-1613353002,NaN,1,2021-02-15,2021-02-15 01:36:42,Direct,(direct),(none),...,0,0,www.fwd.com.sg/,www.fwd.com.sg/fire-insurance/online-quote/qui...,"Online Insurance Company, Insurance Claims | F...",HDB Fire Plan - Instant Quote Online | FWD Sin...,NaN,NaN,NaN,NaN
1215044,1.709967e+18,GA1.3.398132808.1614584564,1709967391439231732-1614599755,NaN,3,2021-03-01,2021-03-01 11:55:55,Organic Search,google,organic,...,0,0,www.fwd.com.sg/travel-insurance/,www.fwd.com.sg/travel-insurance/online-quote/q...,Buy Travel Insurance Plan Online | FWD Singapore,Travel Insurance Online Quotes | FWD Singapore,NaN,NaN,NaN,NaN
1215045,1.545789e+18,GA1.3.359907117.1614676888,1545789298727322520-1614676889,NaN,2,2021-03-02,2021-03-02 09:21:29,Display,dfa,cpm,...,0,0,www.fwd.com.sg/life-insurance/enhanced-term-li...,www.fwd.com.sg/life-insurance/enhanced-term-li...,Buy Term Life Insurance Online | FWD Singapore,Buy Term Life Insurance Online | FWD Singapore,NaN,NaN,NaN,NaN


In [4]:
basicDataExploration(df_session_small)

General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215047 entries, 0 to 1215046
Data columns (total 36 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   full_visitor_id              1215047 non-null  float64
 1   client_id                    1215047 non-null  object 
 2   session_id                   1215047 non-null  object 
 3   quotation_id                 383995 non-null   object 
 4   visit_number                 1215047 non-null  int64  
 5   session_date                 1215047 non-null  object 
 6   session_date_time            1215047 non-null  object 
 7   channelgrouping              1215047 non-null  object 
 8   source                       1215047 non-null  object 
 9   medium                       1215047 non-null  object 
 10  keyword                      1215047 non-null  object 
 11  adcontent                    1215047 non-null  object 
 12  referralpath              

In [5]:
my_value_counts(df_session_small, 'channelgrouping')

,Frequency,Proportion
Paid Search,383276,0.315400
Direct,254667,0.209600
(Other),244026,0.200800
Organic Search,243854,0.200700
Display,49781,0.041000
Referral,37842,0.031100
Social,831,0.000700
Email,770,0.000600
Total,1215047,1.000000


In [6]:
labels = pd.read_csv('Labels.csv')
labels = labels.drop(labels.columns[0], axis=1) 
actual_product = dict(labels.groupby('full_visitor_id')['Label'].first())
actual_product

{701696772471: 'Maid',
 58718125224745: 'Term Life',
 128464091394428: 'Car',
 204704047883456: 'Maid',
 279810159306767: 'HDB Fire',
 313160582548190: 'Term Life',
 337727779205457: 'HDB Fire',
 349861077979131: 'Personal Accident',
 369068162108137: 'Maid',
 509930205731778: 'HDB Fire',
 521470787859847: 'Term Life',
 523034143938632: 'Term Life',
 523656929343719: 'Car',
 529193126440890: 'HDB Fire',
 620310759647846: 'Car',
 663428038908082: 'HDB Fire',
 666215475263557: 'Maid',
 686509196802090: 'Car',
 688648085979660: 'HDB Fire',
 692912989579196: 'Travel',
 702709813945954: 'Term Life',
 725528979510332: 'Car',
 966060027955771: 'Travel',
 1003585151948079: 'HDB Fire',
 1183170621823441: 'Personal Accident',
 1218612696524046: 'Car',
 1238541322757284: 'Car',
 1403352123170670: 'Big 3 Critical Illness',
 1523731468131348: 'Home',
 1615493434633563: 'Personal Accident',
 1620467013104584: 'International Health',
 1673625828169851: 'Maid',
 1818658280793988: 'Term Life',
 1865086

In [7]:
df_transformed = pd.read_csv('transformed_sessions.csv')
df_transformed

,full_visitor_id,quotation_id,visit_number,channelgrouping,browser,operatingsystem,devicecategory,session_quality_dim,bounces,total_pageviews,...,total_pa_pageviews,applicant_drop_out_flag,Entry Product,Exit Product,product_type,Quotation,number_of_quotation,avg_pageview,Num_of_Sessions,is_last_session
0,701696772471,NaN,1,(Other),Safari,iOS,mobile,21,0,3,...,0,0,Maid,Maid,NaN,1.0,2,7.500000,4,NaN
1,701696772471,NaN,4,(Other),Safari,iOS,mobile,52,0,9,...,0,0,Maid,Maid,NaN,1.0,2,7.500000,4,1.0
2,701696772471,8O3Z1JIIZ,4,(Other),Safari,iOS,mobile,52,0,9,...,0,0,Maid,Maid,Maid,1.0,2,7.500000,4,1.0
3,701696772471,CETP1PS8B,4,(Other),Safari,iOS,mobile,52,0,9,...,0,0,Maid,Maid,NaN,1.0,2,7.500000,4,1.0
4,58718125224745,NaN,1,Display,Safari,Macintosh,desktop,1,1,1,...,0,0,Term Life,Term Life,NaN,0.0,0,1.000000,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479770,8557612069918947328,NaN,1,(Other),Safari,iOS,mobile,22,0,7,...,0,0,Term Life,Term Life,NaN,0.0,0,4.000000,2,NaN
479771,8765145035675862016,NaN,1,(Other),Samsung Internet,Android,mobile,4,0,17,...,0,0,Term Life,Term Life,NaN,1.0,1,7.600000,2,NaN
479772,8895526692170194944,NaN,1,(Other),Chrome,Windows,desktop,2,0,2,...,0,0,Term Life,Term Life,NaN,0.0,0,1.500000,2,NaN
479773,9103898263197699072,NaN,4,Referral,Chrome,Android,mobile,22,0,3,...,0,0,Motor,Motor,NaN,0.0,0,2.333333,4,1.0


In [7]:
df_temp = df_transformed.groupby('full_visitor_id')['visit_number'].nunique()
repeat_visitors = df_temp[df_temp>1].index

df_session_small = df_transformed[df_transformed['full_visitor_id'].isin(repeat_visitors)]

# Remove the latest sessions from the dataset
test_dict = dict(df_transformed.groupby(['full_visitor_id'])['visit_number'].max())

# Get Number of Session
df_transformed['Num_of_Sessions'] = df_transformed['full_visitor_id'].map(test_dict)
df_transformed.loc[(df_transformed['visit_number'] == df_transformed['Num_of_Sessions']),'is_last_session'] = 1
df_transformed = df_transformed[df_transformed['is_last_session'] != 1]

# Create Final Dataset
df_rulebased = df_transformed[['full_visitor_id','Exit Product']]
df_rulebased['label'] = df_transformed['full_visitor_id'].map(actual_product)
df_rulebased = df_rulebased.rename(columns = {'Exit Product':'Prediction'})
df_rulebased

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,full_visitor_id,Prediction,label
0,701696772471,Maid,Maid
4,58718125224745,Term Life,Term Life
6,128464091394428,Car,Car
7,128464091394428,Car,Car
9,204704047883456,Maid,Maid
...,...,...,...
479765,8382308323980873728,Personal Accident,Personal Accident
479766,8382308323980873728,Personal Accident,Personal Accident
479770,8557612069918947328,Term Life,Term Life
479771,8765145035675862016,Term Life,Term Life


In [8]:
df_rulebased['is_correct'] = np.where(df_rulebased['Prediction'] == df_rulebased['label'], 'True', 'False')
df_rulebased

,full_visitor_id,Prediction,label,is_correct
0,701696772471,Maid,Maid,True
4,58718125224745,Term Life,Term Life,True
6,128464091394428,Car,Car,True
7,128464091394428,Car,Car,True
9,204704047883456,Maid,Maid,True
...,...,...,...,...
479765,8382308323980873728,Personal Accident,Personal Accident,True
479766,8382308323980873728,Personal Accident,Personal Accident,True
479770,8557612069918947328,Term Life,Term Life,True
479771,8765145035675862016,Term Life,Term Life,True


In [9]:
df_rulebased['is_correct'].value_counts()/len(df_rulebased)

True     0.840028
False    0.159972
Name: is_correct, dtype: float64

In [19]:
df_rulebased.to_csv("rulebased_prediction.csv", index=False)

In [31]:
df_true = df_rulebased.loc[df_rulebased['is_correct'] == 'True']
df_true

,full_visitor_id,Prediction,label,is_correct
0,701696772471,Maid,Maid,True
4,58718125224745,Term Life,Term Life,True
6,128464091394428,Car,Car,True
7,128464091394428,Car,Car,True
9,204704047883456,Maid,Maid,True
...,...,...,...,...
479765,8382308323980873728,Personal Accident,Personal Accident,True
479766,8382308323980873728,Personal Accident,Personal Accident,True
479770,8557612069918947328,Term Life,Term Life,True
479771,8765145035675862016,Term Life,Term Life,True


In [14]:
df_false = df_rulebased.loc[df_rulebased['is_correct'] == 'False']
df_false

,full_visitor_id,Prediction,label,is_correct
19,337727779205457,Home,HDB Fire,False
20,337727779205457,Home,HDB Fire,False
21,337727779205457,Home,HDB Fire,False
23,349861077979131,Travel,Personal Accident,False
63,692912989579196,Maid,Travel,False
...,...,...,...,...
479739,6095257119539420160,Term Life,HDB Fire,False
479740,6095257119539420160,Term Life,HDB Fire,False
479743,6184800977139810304,Term Life,Big 3 Critical Illness,False
479754,7405379267615574016,Term Life,Big 3 Critical Illness,False


In [30]:
my_value_counts(df_false, 'label')

,Frequency,Proportion
Car,7566,0.166000
Home,5312,0.116500
Term Life,4658,0.102200
Big 3 Critical Illness,4388,0.096300
HDB Fire,4195,0.092000
Personal Accident,3615,0.079300
Cancer,2566,0.056300
Travel,2519,0.055300
Maid,2416,0.053000
Motor,2379,0.052200


In [32]:
my_value_counts(df_true, 'label')

,Frequency,Proportion
Car,62729,0.262000
HDB Fire,29354,0.122600
Maid,27530,0.115000
Home,22262,0.093000
Motor,19483,0.081400
Term Life,17376,0.072600
Big 3 Critical Illness,16637,0.069500
Travel,11979,0.050000
Personal Accident,10569,0.044200
Cancer,7990,0.033400


In [19]:
df_false['label'].value_counts()/df_rulebased['label'].value_counts()

Big 3 Critical Illness    0.208704
Cancer                    0.243085
Car                       0.107632
Commercial Car            0.324790
Covid-19                  0.328718
HDB Fire                  0.125041
Heart Attack              0.586806
Home                      0.192645
International Health      0.241130
Maid                      0.080679
Motor                     0.108819
Personal Accident         0.254865
Stroke                    0.452489
Term Life                 0.211401
Travel                    0.173748
Name: label, dtype: float64

## Home - HDB Fire Analysis

In [ ]:
df_home = df_session_small.loc[(df_session_small['Entry Product'] == 'Home')]
df_home

In [ ]:
homevisitor_list = df_home['full_visitor_id'].unique()
homevisitor_list

In [ ]:
df_hdbfire = df_session_small.loc[(df_session_small['Entry Product'] == 'HDB Fire')]
df_hdbfire

In [ ]:
df_hdb = df_hdbfire['full_visitor_id'].isin(homevisitor_list)
df_hdb.value_counts()/len(df_hdb)

In [ ]:
df_hdb = df_hdbfire['full_visitor_id'].isin(homevisitor_list)
df_hdb.value_counts()/len(df_hdb)

In [ ]:
my_value_counts(df_home, 'Exit Product')

In [ ]:
my_value_counts(df_homehdb, 'Exit Product')

## Product Distribution

In [11]:
df_maid = df_session_small.loc[(df_session_small['Entry Product'] == 'Maid')]
df_life = df_session_small.loc[(df_session_small['Entry Product'] == 'Term Life')]
df_car = df_session_small.loc[(df_session_small['Entry Product'] == 'Car')]
df_hdb = df_session_small.loc[(df_session_small['Entry Product'] == 'HDB Fire')]
df_travel = df_session_small.loc[(df_session_small['Entry Product'] == 'Travel')]
df_accident = df_session_small.loc[(df_session_small['Entry Product'] == 'Personal Accident')]
df_big3 = df_session_small.loc[(df_session_small['Entry Product'] == 'Big 3 Critical Illness')]
df_home = df_session_small.loc[(df_session_small['Entry Product'] == 'Home')]
df_health = df_session_small.loc[(df_session_small['Entry Product'] == 'International Health')]
df_motor = df_session_small.loc[(df_session_small['Entry Product'] == 'Motor')]
df_covid = df_session_small.loc[(df_session_small['Entry Product'] == 'Covid-19')]
df_cancer = df_session_small.loc[(df_session_small['Entry Product'] == 'Cancer')]
df_comcar = df_session_small.loc[(df_session_small['Entry Product'] == 'Commercial Car')]
df_stroke = df_session_small.loc[(df_session_small['Entry Product'] == 'Stroke')]
df_heart = df_session_small.loc[(df_session_small['Entry Product'] == 'Heart Attack')]

In [12]:
my_value_counts(df_maid, 'Exit Product')

,Frequency,Proportion
Maid,53392,0.982600
Car,170,0.003100
Home,123,0.002300
International Health,121,0.002200
Covid-19,118,0.002200
Big 3 Critical Illness,101,0.001900
Personal Accident,93,0.001700
Travel,51,0.000900
Term Life,43,0.000800
Cancer,41,0.000800


In [13]:
my_value_counts(df_life, 'Exit Product')

,Frequency,Proportion
Term Life,32268,0.952100
Big 3 Critical Illness,507,0.015000
Personal Accident,292,0.008600
Cancer,155,0.004600
Car,145,0.004300
International Health,122,0.003600
Home,74,0.002200
Motor,69,0.002000
Maid,67,0.002000
Covid-19,53,0.001600


In [14]:
my_value_counts(df_car, 'Exit Product')

,Frequency,Proportion
Car,115373,0.959400
Motor,1398,0.011600
Commercial Car,671,0.005600
Personal Accident,453,0.003800
Home,397,0.003300
Maid,388,0.003200
Travel,360,0.003000
Big 3 Critical Illness,335,0.002800
HDB Fire,257,0.002100
International Health,227,0.001900


In [15]:
my_value_counts(df_hdb, 'Exit Product')

,Frequency,Proportion
HDB Fire,51894,0.942800
Home,2795,0.050800
Personal Accident,93,0.001700
Car,51,0.000900
Big 3 Critical Illness,41,0.000700
Motor,28,0.000500
Covid-19,26,0.000500
Travel,25,0.000500
Maid,22,0.000400
Cancer,21,0.000400


In [16]:
my_value_counts(df_travel, 'Exit Product')

,Frequency,Proportion
Travel,22327,0.922400
Car,386,0.015900
Maid,359,0.014800
Covid-19,279,0.011500
Personal Accident,273,0.011300
Home,136,0.005600
International Health,135,0.005600
HDB Fire,82,0.003400
Big 3 Critical Illness,73,0.003000
Motor,68,0.002800


In [17]:
my_value_counts(df_accident, 'Exit Product')

,Frequency,Proportion
Personal Accident,22532,0.951000
Car,360,0.015200
Big 3 Critical Illness,132,0.005600
Motor,115,0.004900
International Health,101,0.004300
Covid-19,82,0.003500
Travel,63,0.002700
Maid,62,0.002600
Home,61,0.002600
Cancer,58,0.002400


In [18]:
my_value_counts(df_big3, 'Exit Product')

,Frequency,Proportion
Big 3 Critical Illness,32684,0.954600
Cancer,563,0.016400
Term Life,326,0.009500
Personal Accident,215,0.006300
International Health,72,0.002100
Home,67,0.002000
Heart Attack,64,0.001900
Car,63,0.001800
Stroke,39,0.001100
Covid-19,34,0.001000


In [19]:
my_value_counts(df_home, 'Exit Product')

,Frequency,Proportion
Home,47245,0.928700
HDB Fire,2541,0.050000
International Health,251,0.004900
Car,244,0.004800
Personal Accident,128,0.002500
Maid,87,0.001700
Cancer,86,0.001700
Big 3 Critical Illness,85,0.001700
Travel,64,0.001300
Term Life,52,0.001000


In [20]:
my_value_counts(df_health, 'Exit Product')

,Frequency,Proportion
International Health,9341,0.946300
Big 3 Critical Illness,120,0.012200
Personal Accident,119,0.012100
Travel,62,0.006300
Term Life,50,0.005100
Maid,47,0.004800
Covid-19,34,0.003400
Cancer,33,0.003300
Heart Attack,18,0.001800
Home,17,0.001700


In [21]:
my_value_counts(df_motor, 'Exit Product')

,Frequency,Proportion
Motor,35072,0.984000
Car,336,0.009400
Personal Accident,65,0.001800
Commercial Car,43,0.001200
Big 3 Critical Illness,34,0.001000
Maid,18,0.000500
Cancer,16,0.000400
Term Life,15,0.000400
Home,14,0.000400
Travel,12,0.000300


In [22]:
my_value_counts(df_covid, 'Exit Product')

,Frequency,Proportion
Covid-19,7965,0.911800
Travel,326,0.037300
Personal Accident,146,0.016700
Maid,106,0.012100
International Health,57,0.006500
Big 3 Critical Illness,40,0.004600
Term Life,24,0.002700
Home,18,0.002100
Cancer,14,0.001600
Heart Attack,11,0.001300


In [23]:
my_value_counts(df_cancer, 'Exit Product')

,Frequency,Proportion
Cancer,15818,0.930600
Big 3 Critical Illness,518,0.030500
Term Life,185,0.010900
Car,125,0.007400
Personal Accident,114,0.006700
Home,58,0.003400
International Health,32,0.001900
Covid-19,32,0.001900
Heart Attack,26,0.001500
Motor,24,0.001400


In [24]:
my_value_counts(df_comcar, 'Exit Product')

,Frequency,Proportion
Commercial Car,10237,0.951700
Car,366,0.034000
Motor,77,0.007200
Personal Accident,17,0.001600
Cancer,14,0.001300
Home,8,0.000700
Big 3 Critical Illness,7,0.000700
Term Life,7,0.000700
Maid,6,0.000600
Heart Attack,5,0.000500


In [25]:
my_value_counts(df_stroke, 'Exit Product')

,Frequency,Proportion
Stroke,592,0.838500
Big 3 Critical Illness,45,0.063700
Personal Accident,24,0.034000
Car,10,0.014200
Travel,8,0.011300
Heart Attack,6,0.008500
Term Life,4,0.005700
Cancer,4,0.005700
Home,4,0.005700
International Health,3,0.004200


In [26]:
my_value_counts(df_heart, 'Exit Product')

,Frequency,Proportion
Heart Attack,377,0.710000
Big 3 Critical Illness,58,0.109200
Cancer,27,0.050800
Car,13,0.024500
Term Life,13,0.024500
Travel,7,0.013200
Stroke,7,0.013200
Home,7,0.013200
International Health,6,0.011300
Personal Accident,5,0.009400
